<br><font face="Times New Roman" size=5><div dir=ltr align=center>
<font color=blue size=8>
    Introduction to Machine Learning <br>
<font color=red size=5>
    Sharif University of Technology - Computer Engineering Department <br>
    Fall 2022<br> <br>
<font color=black size=6>
    Homework 2: Practical - Linear Regression
    </div>
<br><br>
<font size=4>
   **Name**: Naser Kazemi<br>
   **Student ID**: 99102059 <br> <br>

<font face="Times New Roman" size=4><div dir=ltr>
# Problem 1: Linear Regression Model (40 + 30 optional points)
According to <a href="https://github.com/asharifiz/Introduction_to_Machine_Learning/blob/main/Jupyter_Notebooks/Chapter_02_Classical_Models/Linear%20regression.ipynb"><font face="Roboto">Linear Regression Notebook</font></a>, train a linear regression model on an arbitrary dataset. Explain your chosen dataset and split your data into train and test sets, then predict values for the test set using your trained model. Try to find the best hyperparameters for your model. (Using Lasso Regression, Ridge Regression or Elastic Net and comparing them will have extra optional points)
<br> Explain each step of your workflow.

# Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

# Dataset

Dataset we use here is the insurance dataset. This dataset contains information about the insurance charges of 1338 people. The dataset contains 7 columns, which are: age, sex, bmi, children, smoker, region and charges. The target variable is charges, which is the amount of money the insurance company charges for the person. The other columns are the features of the dataset. The dataset is available in the following link: https://www.kaggle.com/mirichoi0218/insurance

# Encoding

This dataset contains categorical data, so we need to encode them. We use one hot encoding for the categorical data. We use the get_dummies function from pandas to encode the data.

# Splitting the data

We split the data into train and test sets. We use the train_test_split function from sklearn to split the data. We use 20% of the data for testing and 80% for training.

## Load Dataset and inspect it

In [2]:
data = pd.read_csv('insurance.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [3]:
data.describe()

,age,bmi,children,charges
count,1338.000000,1338.000000,1338.000000,1338.000000
mean,39.207025,30.663397,1.094918,13270.422265
std,14.049960,6.098187,1.205493,12110.011237
min,18.000000,15.960000,0.000000,1121.873900
25%,27.000000,26.296250,0.000000,4740.287150
50%,39.000000,30.400000,1.000000,9382.033000
75%,51.000000,34.693750,2.000000,16639.912515
max,64.000000,53.130000,5.000000,63770.428010


In [4]:
data.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


## Data Normalization

We normalize the data to make the training process faster and more stable. We Normalize the data using the following formula:

$$
x_{norm} = \frac{x - \mu}{\sigma}
$$

where $x$ is the data, $\mu$ is the mean of the data, and $\sigma$ is the standard deviation of the data.


In [5]:
categorical_columns = ['sex', 'smoker', 'region']
data_encoded = pd.get_dummies(data, columns=categorical_columns, dtype=np.int8)
data_encoded = data_encoded.rename(columns={"sex_female": "sex", "smoker_yes": "smoker"})\
                .drop(columns=["sex_male", "smoker_no"])

X = data_encoded.drop("charges", axis=1)
y = data_encoded['charges']

X_norm = X.copy()

X_norm[['age', 'bmi']] = (X[['age', 'bmi']] - X[['age', 'bmi']].mean()) / X[['age', 'bmi']].std()
y_norm = (y - y.mean()) / y.std()
X_norm = X_norm.to_numpy()
y_norm = y_norm.to_numpy()

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X_norm, y_norm, test_size=0.2, random_state=42)

# Linear Function

In linear regression, we have a linear function that we want to find its parameters. The linear function is defined as follows:

$$
\hat{y} = w_0 + w_1x_1 + w_2x_2 + ... + w_nx_n
$$

where $w_0$ is the bias, $w_1$ to $w_n$ are the weights, and $x_1$ to $x_n$ are the features.

# Loss Function

In linear regression, we have a loss function that we want to minimize. The loss function is defined as follows:

$$
L = \frac{1}{2N}\sum_{i=1}^{N}(y_i - \hat{y}_i)^2
$$

where $N$ is the number of samples, $y_i$ is the true value of the $i$ th sample, and $\hat{y}_i$ is the predicted value of the $i$ th sample. This loss function is called the Mean Squared Error (MSE).

# Linear Regression Using Normal Equation

The linear function can be rewritten as follows: 

$$
\hat{y} = Xw
$$

where $X$ is the feature with a column of ones added to it, and $w$ is the weight vector with the bias added to it.

The loss function can be rewritten as follows:

$$
L(w) = \frac{1}{2N}(y - Xw)^T(y - Xw)
$$

where $y$ is the true value vector.

The gradient of the loss function with respect to the weight vector can be rewritten as follows:

$$
\frac{\partial L(w)}{\partial w} = \frac{1}{N}X^T(Xw - y)
$$

By setting the gradient to zero, we can find the weight vector that minimizes the loss function, we obtain:

$$
w = (X^TX)^{-1}X^Ty
$$


where $w$ is the vector of weights, $X$ is the matrix of features, and $y$ is the vector of true values.

In [7]:
X_train = np.c_[np.ones(X_train.shape[0]), X_train]
X_test = np.c_[np.ones(X_test.shape[0]), X_test]

In [8]:
# normal equation
w = np.linalg.inv(X_train.T @ X_train) @ X_train.T @ y_train

y_pred = X_test @ w
((y_pred - y_test) ** 2).mean()

0.30160798394871147

# Linear Regression using Gradient Descent


# Gradient Descent





In linear regression using gradient descent algorithm, we want to use to find the best parameters in an iterative manner. The algorithm is defined as follows:

1. Initialize the weight vector with random values.
2. Calculate the gradient of the loss function with respect to the weight vector.
3. Update the weight vector using the gradient by update rule:
$$
w_{n + 1} := w_n - \alpha \frac{\partial L(w_n)}{\partial w}
$$

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Where $L$ is the loss function, $w$ is the weight vector, and $\alpha$ is the learning rate.

4. Repeat steps 2 and 3 until the loss function converges or the maximum number of iterations is reached.

in step 2, we calculate the gradient of the loss function with respect to the weight vector as follows:

$$
\frac{\partial L(w)}{\partial w} = \frac{1}{N}X^T(Xw - y)
$$

In gradient descent algorithm, we have two hyper-parameters: learning rate and maximum number of iterations. The learning rate determines how much we update the weight vector in each iteration. If the learning rate is too small, the algorithm will take a long time to converge. If the learning rate is too large, the algorithm may not converge. The maximum number of iterations determines the maximum number of iterations the algorithm runs. If the algorithm does not converge in the maximum number of iterations, the algorithm stops.

# K-Fold Cross Validation

In K-Fold Cross Validation, we split the data into K folds. Then, we train the model on K-1 folds and test it on the remaining fold. We repeat this process K times, each time using a different fold for testing. Finally, we average the results of the K tests to get the final result.


# Grid Search

Grid search is a method to find the best hyper-parameters for a model. In grid search, we define a set of hyper-parameters and the algorithm tries all the combinations of the hyper-parameters to find the best combination. The best combination is the combination that gives the best performance on the validation set. The performance is measured by a metric, which is usually the accuracy or the loss function. Here we use the loss function as the metric and K-fold cross validation as the evaluation method.

In [9]:
def gradient_descent(X, y, **kwargs):
    learning_rate = kwargs.get('learning_rate', 0.01)
    max_iter = kwargs.get('max_iter', 1000)
    
    n = X.shape[0]
    w = np.array(np.random.rand(X.shape[1]), dtype=np.float64)
    for i in range(max_iter):
        y_pred = X.dot(w)
        gradient = 1/n * X.T.dot(y_pred - y)
        w = w - learning_rate * gradient
    return w


def k_fold_cross_validation(X, y, method, k=5, **kwargs):
    n = X.shape[0]
    indices = np.random.permutation(n) - 1
    X = X[indices]
    y = y[indices]
    fold_size = n // k
    scores = []
    for i in range(k):
        X_test = X[i*fold_size:(i+1)*fold_size]
        y_test = y[i*fold_size:(i+1)*fold_size]
        X_train = np.r_[X[:i*fold_size], X[(i+1)*fold_size:]]
        y_train = np.r_[y[:i*fold_size], y[(i+1)*fold_size:]]
        w = method(X_train, y_train, **kwargs)
        y_pred = X_test @ w
        scores.append(np.mean((y_pred - y_test)**2))
    
    return np.mean(scores)
    

# Train and Evaluate the Model

In [10]:
# train and evaluate

learning_rates = [0.01, 0.05, 0.1, 0.5, 1]
max_iters = [100, 500, 1000, 5000, 10000]

scores = {}
for learning_rate in learning_rates:
    for max_iter in max_iters:
        score = k_fold_cross_validation(X_train, y_train, gradient_descent, learning_rate=learning_rate, max_iter=max_iter)
        scores[(learning_rate, max_iter)] = score
scores

/var/folders/22/1gdchc0s1zv4lr78srz6vdd80000gn/T/ipykernel_38509/2530167079.py:28: RuntimeWarning: overflow encountered in square
  scores.append(np.mean((y_pred - y_test)**2))


{(0.01, 100): 0.683226429062544,
 (0.01, 500): 0.36999569776430696,
 (0.01, 1000): 0.28027812642447963,
 (0.01, 5000): 0.25936297962412946,
 (0.01, 10000): 0.2585200796308765,
 (0.05, 100): 0.3388422385060749,
 (0.05, 500): 0.2598228267206427,
 (0.05, 1000): 0.26097346262853616,
 (0.05, 5000): 0.25915736662152733,
 (0.05, 10000): 0.2631444402765875,
 (0.1, 100): 0.2870741119445859,
 (0.1, 500): 0.25917568282808984,
 (0.1, 1000): 0.25984639798586945,
 (0.1, 5000): 0.2619766650701677,
 (0.1, 10000): 0.2597265341233589,
 (0.5, 100): 0.25899457578442053,
 (0.5, 500): 0.25941834007738507,
 (0.5, 1000): 0.26039241742085034,
 (0.5, 5000): 0.2601787604944053,
 (0.5, 10000): 0.2607197497088648,
 (1, 100): 2.6074608896159157e+86,
 (1, 500): inf,
 (1, 1000): nan,
 (1, 5000): nan,
 (1, 10000): nan}

In [11]:
best_params, best_score,  = min(scores.items(), key=lambda x: x[1])
best_params, best_score

((0.01, 10000), 0.2585200796308765)

In [12]:
w = gradient_descent(X_test, y_test, learning_rate=best_params[0], max_iter=best_params[1])
y_pred = X_test @ w
((y_pred - y_test) ** 2).mean(), w

(0.22247973395103585,
 array([-1.01034545,  0.293039  ,  0.17965188,  0.06032784,  0.03913548,
         2.04812882,  0.59708199,  0.56121591,  0.38134379,  0.45198632]))

## Lasso Regression

Lasso regression is a linear regression model that uses L1 regularization. Lasso regression adds the following term to the loss function:

$$
\frac{\lambda}{N}\sum_{i=1}^{n}|w_i|
$$

where $\lambda$ is the regularization parameter. The regularization parameter determines how much we penalize the weights. If the regularization parameter is zero, the loss function is the same as the loss function of linear regression. If the regularization parameter is large, the residual sum of squares in linear regression decreases,variance of model decreases and bias increases.


The gradient of lasso cost function with respect to the weight vector can be rewritten as follows:

$$
\frac{\partial L(w)}{\partial w} = \frac{1}{N}(X^T(Xw - y) + \lambda sign(w))
$$

where $sign(w)$ is the sign function that returns 1 if $w$ is positive, -1 if $w$ is negative, and 0 if $w$ is zero.




In [13]:
def lasso_regression(X, y, **kwargs):
    learning_rate = kwargs.get('learning_rate', 0.01)
    max_iter = kwargs.get('max_iter', 1000)
    lambda_ = kwargs.get('lambda_', 0.1)
    
    n = X.shape[0]
    w = np.array(np.random.rand(X.shape[1]), dtype=np.float64)
    for i in range(max_iter):
        y_pred = X.dot(w)
        gradient = 1/n * (X.T.dot(y_pred - y) + lambda_ * np.sign(w))
        w = w - learning_rate * gradient
    return w

In [14]:
learning_rates = [0.01, 0.05, 0.1, 0.5, 1]
max_iters = [100, 500, 1000, 5000, 10000]
lambdas_ = [0.1, 0.5, 1, 5, 10]
scores = {}

for learning_rate in learning_rates:
    for max_iter in max_iters:
        for lambda_ in lambdas_:
            score = k_fold_cross_validation(X_train, y_train, lasso_regression,
                                            learning_rate=learning_rate, max_iter=max_iter, lambda_=lambda_)
            scores[(learning_rate, max_iter, lambda_)] = score
            

    
    
best_params, best_score = min(scores.items(), key=lambda x: x[1])
best_params, best_score

/var/folders/22/1gdchc0s1zv4lr78srz6vdd80000gn/T/ipykernel_38509/2530167079.py:28: RuntimeWarning: overflow encountered in square
  scores.append(np.mean((y_pred - y_test)**2))


((0.5, 100, 1), 0.257177751606546)

In [15]:
w = lasso_regression(X_test, y_test, learning_rate=best_params[0], max_iter=best_params[1], lambda_=best_params[2])
y_pred = X_test @ w
((y_pred - y_test) ** 2).mean(), w

(0.22299707373622624,
 array([-5.27077179e-01,  2.92122473e-01,  1.70156103e-01,  5.51552476e-02,
         1.75341880e-02,  2.01487129e+00,  1.24237683e-01,  8.92614388e-02,
        -5.54295740e-02, -4.39110828e-05]))

# Ridge Regression

Ridge regression is a linear regression model that uses L2 regularization. Ridge regression adds the following term to the loss function:

$$
\frac{\lambda}{N}\sum_{i=1}^{n}w_i^2
$$

where $\lambda$ is the regularization parameter. The regularization parameter determines how much we penalize the weights. If the regularization parameter is zero, the loss function is the same as the loss function of linear regression. If the regularization parameter is large, the residual sum of squares in linear regression decreases,variance of model decreases and bias increases.

In [16]:
def ridge_regression(X, y, **kwargs):
    learning_rate = kwargs.get('learning_rate', 0.01)
    max_iter = kwargs.get('max_iter', 1000)
    lambda_ = kwargs.get('lambda_', 0.1)
    
    n = X.shape[0]
    w = np.array(np.random.rand(X.shape[1]), dtype=np.float64)
    for i in range(max_iter):
        y_pred = X.dot(w)
        gradient = 1/n * (X.T.dot(y_pred - y) + 2 * lambda_ * w)
        w = w - learning_rate * gradient
    return w

In [17]:
learning_rates = [0.01, 0.05, 0.1, 0.5, 1]
max_iters = [100, 500, 1000, 5000, 10000]
lambdas_ = [0.1, 0.5, 1, 5, 10]
scores = {}

for learning_rate in learning_rates:
    for max_iter in max_iters:
        for lambda_ in lambdas_:
            score = k_fold_cross_validation(X_train, y_train, ridge_regression,
                                            learning_rate=learning_rate, max_iter=max_iter, lambda_=lambda_)
            scores[(learning_rate, max_iter, lambda_)] = score
    
    
best_params, best_score = min(scores.items(), key=lambda x: x[1])
best_params, best_score

/var/folders/22/1gdchc0s1zv4lr78srz6vdd80000gn/T/ipykernel_38509/2530167079.py:28: RuntimeWarning: overflow encountered in square
  scores.append(np.mean((y_pred - y_test)**2))


((0.1, 500, 0.1), 0.25737993166713674)

In [18]:
w = ridge_regression(X_test, y_test, learning_rate=best_params[0], max_iter=best_params[1], lambda_=best_params[2])
y_pred = X_test @ w
((y_pred - y_test) ** 2).mean(), w

(0.22250161728091114,
 array([-0.59204677,  0.29334912,  0.17917662,  0.05993538,  0.03699012,
         2.03649347,  0.18231408,  0.14657244, -0.03133775,  0.03769973]))

# Elastic Net Regression

Elastic net regression is a linear regression model that uses L1 and L2 regularization. Elastic net regression adds the following term to the loss function:

$$
\lambda_1\sum_{i=1}^{n}|w_i| + \lambda_2\sum_{i=1}^{n}w_i^2
$$

where $\lambda_1$ and $\lambda_2$ are the regularization parameters. The regularization parameters determine how much we penalize the weights. If the regularization parameters are zero, the loss function is the same as the loss function of linear regression. If the regularization parameters are large, the residual sum of squares in linear regression decreases,variance of model decreases and bias increases.

In [19]:
# elastic net regression
def elastic_net_regression(X, y, **kwargs):
    learning_rate = kwargs.get('learning_rate', 0.01)
    max_iter = kwargs.get('max_iter', 1000)
    lambda_1 = kwargs.get('lambda_1', 0.1)
    lambda_2 = kwargs.get('lambda_2', 0.5)
    
    n = X.shape[0]
    w = np.array(np.random.rand(X.shape[1]), dtype=np.float64)
    for i in range(max_iter):
        y_pred = X.dot(w)
        gradient = 1/n * (X.T.dot(y_pred - y) + 2 * lambda_1 * w + lambda_2 * lambda_ * np.sign(w))
        w = w - learning_rate * gradient
    return w

In [20]:
learning_rates = [0.01, 0.05, 0.1, 0.5, 1]
max_iters = [100, 500, 1000, 5000, 10000]
lambdas_1 = [0.1, 0.5, 1, 5, 10]
lambdas_2 = [0.1, 0.5, 1, 5, 10]
scores = {}

for learning_rate in learning_rates:
    for max_iter in max_iters:
        for lambda_1 in lambdas_1:
            for lambda_2 in lambdas_2:
                score = k_fold_cross_validation(X_train, y_train, elastic_net_regression,
                                                learning_rate=learning_rate, max_iter=max_iter, 
                                                lambda_1=lambda_1, lambda_2=lambda_2)
                scores[(learning_rate, max_iter, lambda_1, lambda_2)] = score
                
best_params, best_score = min(scores.items(), key=lambda x: x[1])
best_params, best_score

/var/folders/22/1gdchc0s1zv4lr78srz6vdd80000gn/T/ipykernel_38509/2530167079.py:28: RuntimeWarning: overflow encountered in square
  scores.append(np.mean((y_pred - y_test)**2))
/var/folders/22/1gdchc0s1zv4lr78srz6vdd80000gn/T/ipykernel_38509/1753620372.py:12: RuntimeWarning: overflow encountered in add
  gradient = 1/n * (X.T.dot(y_pred - y) + 2 * lambda_1 * w + lambda_2 * lambda_ * np.sign(w))


((0.1, 1000, 1, 0.1), 0.25701804632848574)

In [21]:
w = elastic_net_regression(X_test, y_test, learning_rate=best_params[0], max_iter=best_params[1], 
                           lambda_1=best_params[2], lambda_2=best_params[3])
y_pred = X_test @ w
((y_pred - y_test) ** 2).mean(), w

(0.22571386866197526,
 array([-4.02316288e-01,  2.94446784e-01,  1.67984800e-01,  5.08820537e-02,
         2.37653260e-03,  1.91863342e+00,  2.30692321e-02,  4.34366607e-04,
        -1.39451573e-01, -8.33561109e-02]))

# Comparing Lasso, Ridge, and Elastic Net Regression

In this section, we compare the performance of lasso, ridge, and elastic net regression on the insurance dataset. As you can see, Lasso regression has the lowest loss function, but the difference between the loss functions of lasso, ridge, and elastic net regression is not significant. Therefore, we can conclude that the performance of lasso, ridge, and elastic net regression is similar on this dataset.

Lasso regression results in more sparse weight vectors than ridge regression and elastic net regression. This means that lasso regression is more likely to remove features from the model than ridge regression and elastic net regression. This is because lasso regression uses L1 regularization, which is more likely to make the weights zero than L2 regularization.

# Conclusion

In this notebook, we have implemented linear regression using normal equation and gradient descent algorithm. We have also implemented lasso, ridge, and elastic net regression. We have used K-fold cross validation and grid search to find the best hyper-parameters for the models. We have used the Boston Housing dataset to train and evaluate the models. The results show that the models perform well on the dataset.